<a href="https://colab.research.google.com/github/Vishy-A/DeepLearning/blob/main/4106HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim
from google.colab import drive
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
from collections import OrderedDict
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from torch.utils.data import DataLoader


In [15]:
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((0.4915, 0.4823, 0.4468),
                           (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified
Files already downloaded and verified


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA (GPU) is available.")
else:
    device = torch.device("cpu")
    print("CUDA (GPU) is not available. Using CPU.")

CUDA (GPU) is available.


In [18]:
def trainingloop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('Epoch {}, Training loss {}'.format(epoch, loss_train / len(train_loader)))

In [19]:
def validateloop(model, train_loader, val_loader):
      for name, loader in [("train", train_loader), ("val", val_loader)]:
          correct = 0
          total = 0



          alllabels = []
          allpreds = []

          with torch.no_grad():
              for imgs, labels in loader:
                  outputs = model(imgs)
                  _, predicted = torch.max(outputs, dim=1)
                  total += labels.shape[0]
                  correct += int((predicted == labels).sum())

                  alllabels.extend(labels.cpu().numpy())
                  allpreds.extend(predicted.cpu().numpy())

          print("Accuracy {}: {:.2f}".format(name , correct / total))

          alllabels = np.array(alllabels)
          allpreds = np.array(allpreds)

          report = classification_report(alllabels, allpreds)
          print(f"Classification Report {name}:\n{report}")



In [20]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return torch.softmax(x, dim=1)

In [21]:
trainloader = DataLoader(cifar10, batch_size=64, shuffle=True)
valloader = DataLoader(cifar10_val, batch_size=64, shuffle=False)

In [22]:
model = Network()
lossfunc = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.001)
epochsReported = [1, 10, 20, 50, 100, 200, 500]

In [23]:
trainingloop(
    n_epochs = 500,
    optimizer = optimizer,
    model = model,
    loss_fn = lossfunc,
    train_loader = valloader,
)

Epoch 1, Training loss 2.189425315067267
Epoch 10, Training loss 2.150033038892564
Epoch 20, Training loss 2.2035691905173524
Epoch 30, Training loss 2.203048684794432
Epoch 40, Training loss 2.1981200397394267
Epoch 50, Training loss 2.24719172678176
Epoch 60, Training loss 2.2471733731069383
Epoch 70, Training loss 2.265219462145666
Epoch 80, Training loss 2.276243660860001
Epoch 90, Training loss 2.2788257811479506
Epoch 100, Training loss 2.2788258054453854
Epoch 110, Training loss 2.2788258054453854
Epoch 120, Training loss 2.2788258054453854
Epoch 130, Training loss 2.2788258054453854
Epoch 140, Training loss 2.2788258054453854
Epoch 150, Training loss 2.342915396781484
Epoch 160, Training loss 2.351178512451755
Epoch 170, Training loss 2.3605297568497385
Epoch 180, Training loss 2.3390366499591027
Epoch 190, Training loss 2.338738225827551
Epoch 200, Training loss 2.338738225827551
Epoch 210, Training loss 2.338738225827551
Epoch 220, Training loss 2.338738225827551
Epoch 230, T

In [24]:
validateloop(model, trainloader, valloader)

Accuracy train: 0.12
Classification Report train:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      5000
           1       0.00      0.00      0.00      5000
           2       0.00      0.00      0.00      5000
           3       0.00      0.00      0.00      5000
           4       0.14      0.56      0.22      5000
           5       0.10      0.59      0.17      5000
           6       0.00      0.00      0.00      5000
           7       0.00      0.00      0.00      5000
           8       0.00      0.00      0.00      5000
           9       0.00      0.00      0.00      5000

    accuracy                           0.12     50000
   macro avg       0.02      0.12      0.04     50000
weighted avg       0.02      0.12      0.04     50000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy val: 0.12
Classification Report val:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1000
           1       0.00      0.00      0.00      1000
           2       0.00      0.00      0.00      1000
           3       0.00      0.00      0.00      1000
           4       0.15      0.58      0.23      1000
           5       0.11      0.64      0.18      1000
           6       0.00      0.00      0.00      1000
           7       0.00      0.00      0.00      1000
           8       0.00      0.00      0.00      1000
           9       0.00      0.00      0.00      1000

    accuracy                           0.12     10000
   macro avg       0.03      0.12      0.04     10000
weighted avg       0.03      0.12      0.04     10000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
class Network2(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 32 * 3, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return torch.softmax(x, dim=1)

In [26]:
newNetwork = Network2()
optimizer_expanded = optim.Adam(newNetwork.parameters(), lr=.001)

trainingloop(500, optimizer_expanded, newNetwork, lossfunc, trainloader)
validateloop(newNetwork, trainloader, valloader)

Epoch 1, Training loss 4.727308712956851
Epoch 10, Training loss 4.729854382517392
Epoch 20, Training loss 4.765280632106849
Epoch 30, Training loss 4.765220689651606
Epoch 40, Training loss 4.765340574562092
Epoch 50, Training loss 4.765280632106849
Epoch 60, Training loss 4.765220689651606
Epoch 70, Training loss 4.765460459472578
Epoch 80, Training loss 4.765400517017335
Epoch 90, Training loss 4.765340574562092
Epoch 100, Training loss 4.765340574562092


KeyboardInterrupt: 